In [1]:
import pandas as pd
import csv
from nltk.tokenize import sent_tokenize,word_tokenize,RegexpTokenizer
import numpy as np
from sklearn import preprocessing

In [2]:
df_train=pd.read_csv('data/train.tsv', sep='\t')
df_train[:30]

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [3]:
df_test=pd.read_csv('data/test.tsv', sep='\t')
df_test[:30]

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine
5,156066,8545,intermittently pleasing but
6,156067,8545,intermittently pleasing
7,156068,8545,intermittently
8,156069,8545,pleasing
9,156070,8545,but


In [13]:
sentences=df_train['Phrase'].tolist()
sentences.extend(df_test['Phrase'].tolist())
text=''
maxlen=0
for sentence in sentences:
    if len(sentence)>maxlen:
        maxlen=len(sentence)
    for s in sentence.lower():
        text+=s
chars=set(text)
vocab_size=len(chars)
char_indices=dict((c,i) for i,c in enumerate(chars))
indices_char=dict((i,c) for i,c in enumerate(chars))
print(maxlen)

283


In [45]:
def vectorize_sentence(sentence,char_indices,length=maxlen):
    X=[]
    x=[char_indices[w] for w in sentence.lower()]
    if len(X)<length:
        x.extend([0 for i in range(length-len(x))])
    x2=np.eye(len(char_indices))[x]
    X.append(x2)
    return x2.reshape((1,x2.shape[0],x2.shape[1]))

In [ ]:
import progressbar

train_phrases=df_train['Phrase'].tolist()
test_phrases=df_test['Phrase'].tolist()
X_train=np.zeros(shape=(0,maxlen,vocab_size))
bar=progressbar.ProgressBar(maxval=len(train_phrases)+1,widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
for item in train_phrases:
    #print(item)
    X_train=np.vstack((X_train,vectorize_sentence(item,char_indices,maxlen)))
    i+=1
    bar.update(i)
bar.finish()
X_test=np.zeros(shape=(0,maxlen,vocab_size))
bar=progressbar.ProgressBar(maxval=len(test_phrases)+1,widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
i=0
for item in test_phrases:
    X_test=np.vstack((X_test,vectorize_sentence(item,char_indices,maxlen)))
    i+=1
    bar.update(i)
bar.finish()
train_labels=df_train['Sentiment'].tolist()
unique_labels=list(set(train_labels))
lb=preprocessing.LabelBinarizer()
lb.fit(unique_labels)
y_train=lb.transform(train_labels)

In [ ]:
print(X_train.shape)
print(df_train.shape)

In [65]:
#read glove data
glove_file='../glove/glove.6B.50d.txt'
pretrained_vectors=pd.read_table(glove_file, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)
base_vector=pretrained_vectors('this')
def vec(w):
    try:
        location=pretrained_vectors.loc[w]
        return location.as_matrix()
    except KeyError:
        return None

TypeError: 'DataFrame' object is not callable

In [52]:
text=df_train['Phrase'][0:1].tolist()[0]
sentences=sent_tokenize(text)
numwords=0
average=vec('this')*0
for sentence in sentences:
    for word in word_tokenize(sentence):
        average+=vec(word.lower())
        numwords+=1
average/=numwords

In [47]:
pretrained_vectors

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
the,0.418000,0.249680,-0.412420,0.121700,0.345270,-0.044457,-0.496880,-0.178620,-0.000660,-0.656600,...,-0.298710,-0.157490,-0.347580,-0.045637,-0.442510,0.187850,0.002785,-0.184110,-0.115140,-0.785810
",",0.013441,0.236820,-0.168990,0.409510,0.638120,0.477090,-0.428520,-0.556410,-0.364000,-0.239380,...,-0.080262,0.630030,0.321110,-0.467650,0.227860,0.360340,-0.378180,-0.566570,0.044691,0.303920
.,0.151640,0.301770,-0.167630,0.176840,0.317190,0.339730,-0.434780,-0.310860,-0.449990,-0.294860,...,-0.000064,0.068987,0.087939,-0.102850,-0.139310,0.223140,-0.080803,-0.356520,0.016413,0.102160
of,0.708530,0.570880,-0.471600,0.180480,0.544490,0.726030,0.181570,-0.523930,0.103810,-0.175660,...,-0.347270,0.284830,0.075693,-0.062178,-0.389880,0.229020,-0.216170,-0.225620,-0.093918,-0.803750
to,0.680470,-0.039263,0.301860,-0.177920,0.429620,0.032246,-0.413760,0.132280,-0.298470,-0.085253,...,-0.094375,0.018324,0.210480,-0.030880,-0.197220,0.082279,-0.094340,-0.073297,-0.064699,-0.260440
and,0.268180,0.143460,-0.278770,0.016257,0.113840,0.699230,-0.513320,-0.473680,-0.330750,-0.138340,...,-0.069043,0.368850,0.251680,-0.245170,0.253810,0.136700,-0.311780,-0.632100,-0.250280,-0.380970
in,0.330420,0.249950,-0.608740,0.109230,0.036372,0.151000,-0.550830,-0.074239,-0.092307,-0.328210,...,-0.486090,-0.008027,0.031184,-0.365760,-0.426990,0.421640,-0.116660,-0.507030,-0.027273,-0.532850
a,0.217050,0.465150,-0.467570,0.100820,1.013500,0.748450,-0.531040,-0.262560,0.168120,0.131820,...,0.138130,0.369730,-0.642890,0.024142,-0.039315,-0.260370,0.120170,-0.043782,0.410130,0.179600
"""",0.257690,0.456290,-0.769740,-0.376790,0.592720,-0.063527,0.205450,-0.573850,-0.290090,-0.136620,...,0.030498,-0.395430,-0.385150,-1.000200,0.087599,-0.310090,-0.346770,-0.314380,0.750040,0.970650


In [81]:
def get_average_vector(review):
    numwords=0.001
    average=np.zeros(base_vector.shape)
    tokenizer = RegexpTokenizer(r'\w+')
    for word in tokenizer.tokenize(review):
    #sentences=sent_tokenize(review)
    #for sentence in sentences:
    #    for word in word_tokenize(sentence):
        value=vec(word.lower())
        if value is not None:
            average+=value
            numwords+=1
        #else:
            #print(word+" not in vectors")
    average/=numwords
    return average.tolist()

In [82]:
df_train['Vectorized_review']=df_train['Phrase'].apply(lambda x:get_average_vector(x))

In [83]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment,Vectorized_review
0,1,1,A series of escapades demonstrating the adage ...,1,"[0.3946884660438273, 0.245754149881432, -0.278..."
1,2,1,A series of escapades demonstrating the adage ...,2,"[0.2717263052639098, 0.2867380901364188, -0.41..."
2,3,1,A series,2,"[0.25190404797601196, 0.5057421289355323, -0.3..."
3,4,1,A,2,"[0.21683316683316686, 0.46468531468531477, -0...."
4,5,1,series,2,"[0.28672327672327674, 0.5462937062937063, -0.3..."


In [97]:
X_train=np.array(df_train['Vectorized_review'].tolist())
train_labels=df_train['Sentiment'].tolist()
unique_labels=list(set(train_labels))
lb=preprocessing.LabelBinarizer()
lb.fit(unique_labels)
y_train=lb.transform(train_labels)

In [98]:
print(X_train.shape)
print(y_train.shape)

(156060, 50)
(156060, 5)


In [99]:
lb.classes_

array([0, 1, 2, 3, 4])

In [93]:
len(X_train)

156060

In [89]:
a = np.zeros((10, 2))
a.shape

(10, 2)

In [90]:
a

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])